# Library Recommender

## Import libraries and data

In [1]:
import pandas as pd

In [ ]:
repo_url = "https://raw.githubusercontent.com/baertsch/MGT-502-ML-Project/main/"

interactions_train_df = pd.read_csv(repo_url + "kaggle_data/interactions_train.csv")
items_df = pd.read_csv(repo_url + "kaggle_data/items.csv")
sample_submission_df = pd.read_csv(repo_url + "kaggle_data/sample_submission.csv")

,user_id,recommendation
0,0,3758 11248 9088 9895 5101 6074 9295 14050 1096...
1,1,3263 726 1589 14911 6432 10897 6484 7961 8249 ...
2,2,13508 9848 12244 2742 11120 2893 2461 5439 116...
3,3,2821 10734 6357 5934 2085 12608 12539 10551 10...
4,4,12425 219 11602 1487 14178 489 13888 2110 4413...
...,...,...
7833,7833,3915 10372 1921 6260 12503 6047 10331 11549 14...
7834,7834,2136 12498 10439 7427 5788 129 12770 2928 8122...
7835,7835,8540 2441 5690 12085 9514 1447 2355 2761 13437...
7836,7836,10948 11251 850 5048 6032 10357 6566 4402 7154...
